### `ExecutorPreloadHandle` Demo

This notebook is dedicated to developers wanting to enhance their
data store implementation by the new _data store preload API_. 
This API has been added to `xcube.core.store.DataStore` in xcube 1.8.
Demonstrated here is the usage of the utility class ` xcube.core.store.preload.ExecutorPreloadHandle` 
for cases where the preload process can be concurrently performed for each indiviual data resource. 

In [1]:
import random
import time

from xcube.core.store.preload import ExecutorPreloadHandle
from xcube.core.store.preload import PreloadHandle
from xcube.core.store.preload import PreloadState
from xcube.core.store.preload import PreloadStatus

In [2]:
data_ids = (
    "tt-data/tinky-winky.nc", 
    "tt-data/dipsy.zarr", 
    "tt-data/laa-laa.tiff", 
    "tt-data/po.zarr.zip"
)

In [3]:
def preload_data(handle: PreloadHandle, data_id: str):
    duration = random.randint(5, 15) # seconds
    num_ticks = 100  
    for i in range(num_ticks):
        time.sleep(duration / num_ticks)
        if handle.cancelled:
            # TODO: Note clear, why future.cancel() doesn't do the job
            handle.notify(PreloadState(data_id, status=PreloadStatus.cancelled))
            return
        handle.notify(PreloadState(data_id, progress=i / num_ticks))
        if i % 10 == 0:
            handle.notify(PreloadState(data_id, message=f"Step #{i // 10 + 1}"))
    handle.notify(PreloadState(data_id, progress=1.0, message="Done."))

---
Synchronous / blocking call

In [4]:
handle = ExecutorPreloadHandle(data_ids, preload_data=preload_data)

In [5]:
handle

Data ID,Status,Progress,Message,Exception
tt-data/tinky-winky.nc,STOPPED,100%,Done.,-
tt-data/dipsy.zarr,STOPPED,100%,Done.,-
tt-data/laa-laa.tiff,STOPPED,100%,Done.,-
tt-data/po.zarr.zip,STOPPED,100%,Done.,-


In [6]:
handle.close()

---
Asynchronous / non-blocking call

In [7]:
async_handle = ExecutorPreloadHandle(data_ids, blocking=False, preload_data=preload_data)

In [8]:
async_handle.show()

In [9]:
time.sleep(2)

In [10]:
async_handle.get_state("tt-data/dipsy.zarr")

PreloadState(data_id='tt-data/dipsy.zarr', status=PreloadStatus.started, progress=0.17, message='Step #2')

In [11]:
print(async_handle.get_state("tt-data/dipsy.zarr"))

data_id=tt-data/dipsy.zarr, status=STARTED, progress=0.17, message=Step #2


In [12]:
time.sleep(2)

In [13]:
async_handle.cancel()

In [14]:
print(async_handle)

Data ID                 Status    Progress    Message    Exception
----------------------  --------  ----------  ---------  -----------
tt-data/tinky-winky.nc  STARTED   38%         Step #4    -
tt-data/dipsy.zarr      STARTED   35%         Step #4    -
tt-data/laa-laa.tiff    STARTED   55%         Step #6    -
tt-data/po.zarr.zip     STARTED   29%         Step #3    -


In [15]:
async_handle

Data ID,Status,Progress,Message,Exception
tt-data/tinky-winky.nc,CANCELLED,38%,Step #4,-
tt-data/dipsy.zarr,STARTED,35%,Step #4,-
tt-data/laa-laa.tiff,STARTED,55%,Step #6,-
tt-data/po.zarr.zip,STARTED,29%,Step #3,-


In [16]:
async_handle.close()